# 3D Reconstruction

Imports:

In [1]:
import numpy as np
import cv2 as cv
import os
from scipy.optimize import least_squares
from tomlkit import boolean
from tqdm import tqdm
import matplotlib.pyplot as plt

Load dataset:

In [7]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"): # add file types as needed
            img = cv.imread(os.path.join(folder, filename))
            if img is not None:
                img_rgb = img
                images.append(img_rgb)
    return images

folder = "dataset"
images = load_images_from_folder(folder)

In [3]:
# Define the size of the chessboard
chessboard_size = (7, 7)

# Define arrays to save the object points and image points from all the images
object_points = [] # 3D points in real world space
image_points = [] # 2D points in image plane

# Prepare grid and points to display
objp = np.zeros((np.prod(chessboard_size), 3), dtype=np.float32)

# Populate objp with coordinated for each grid point
objp[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# Loop over the images
for image in images:
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv.findChessboardCorners(gray_image, chessboard_size, None)

    # If corners are found, add object points and image points
    if ret == True:
        object_points.append(objp)
        image_points.append(corners)

# Calibrate the camera and save the results
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(object_points, image_points, gray_image.shape[::-1], None, None)

In [13]:
# Set the print options to suppress the scientific notation
np.set_printoptions(suppress=True)

# Save the intrinsic parameters to a txt file
np.savetxt("intrinsic_parameters.txt", mtx, fmt='%.2f')

# Print a success message
print("The intrinsic parameters were successfully saved to intrinsic_parameters.txt in normal format.")

In [13]:
# 1. Detect 2D features in every input image.
def detect_features(images):
    sift = cv.SIFT_create()
    keypoints, descriptors = [], []
    for img in images:
        kp, des = sift.detectAndCompute(img, None)
        keypoints.append(kp)
        descriptors.append(des)
    return keypoints, descriptors

In [14]:
# 2. Match 2D features between images.
def match_features(descriptors):
    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
    matches = []
    for i in range(len(descriptors) - 1):
        matches.append(bf.match(descriptors[i], descriptors[i+1]))
    return matches

In [15]:
# 3. Generate correspondence (fundamental matrix) RANSAC
def generate_correspondence(matches, keypoints):
    fundamental_matrices = []
    for i in range(len(matches) - 1):
        pts1 = np.float32([keypoints[i][m.queryIdx].pt for m in matches[i]]).reshape(-1,1,2)
        pts2 = np.float32([keypoints[i+1][m.trainIdx].pt for m in matches[i]]).reshape(-1,1,2)
        F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)
        fundamental_matrices.append(F)
    return fundamental_matrices

In [16]:
# 3. Generate correspondence (fundamental matrix) RANSAC
def generate_correspondence(matches, keypoints):
    fundamental_matrices = []
    for i in range(len(matches) - 1):
        pts1 = np.float32([keypoints[i][m.queryIdx].pt for m in matches[i]]).reshape(-1,1,2)
        pts2 = np.float32([keypoints[i+1][m.trainIdx].pt for m in matches[i]]).reshape(-1,1,2)
        F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)
        fundamental_matrices.append(F)
    return fundamental_matrices

In [17]:
# Now we can use the functions we defined above
keypoints, descriptors = detect_features(images)
matches = match_features(descriptors)
fundamental_matrices = generate_correspondence(matches, keypoints)